In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Unión conjuntos de datos
data0 = pd.read_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/0_dataset_resumen_c.csv', on_bad_lines='skip', sep=';')
data1 = pd.read_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/lista7.csv', on_bad_lines='skip', sep=';')

df = pd.merge(left=data0,right=data1, left_on='user', right_on='username')
df.info()

In [ ]:
# Sustituimos los NaN por la etiqueta 'Other'
df['category'] = df['category'].fillna('Other')

 **GENERACIÓN CARACTERISTICAS **





In [ ]:
import csv
from datetime import datetime
fechas= []
hora = []
format = []
horaf = []
meses = []
mesf = []



with open('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/0_dataset_resumen_c.csv', 'r') as File:
                        reader = csv.reader(File, delimiter=';')
                        # Omitir el encabezado
                        next(reader, None)
                        for row in reader:
                            
                            fechas.append(datetime.strptime(row[4], '%Y-%m-%d'))
                            hora.append(datetime.strptime(row[5], '%H:%M:%S'))
                      
                        print(fechas)
                        print(hora)
                        for fec in fechas:
                          format.append(fec.strftime('%A'))
                          meses.append(fec.strftime('%m'))
                        print(format)
                        print(meses)
                        for hr in hora:
                          horaf.append(hr.strftime('%H'))
                        print(horaf)
                        
                        df['day'] = format
                        df['month'] = meses
                        df['hour_of_day'] = horaf

df = df.astype({'hour_of_day': float})
df.dtypes
                      

In [ ]:
df

In [ ]:
df.to_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/datos_def.csv')

In [ ]:
dfs = pd.read_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/datos_def.csv', on_bad_lines='skip', sep=',')

In [ ]:
dfs

In [ ]:
#Elimino post de tipo video
data_sin_videos = dfs[dfs['tipo']== 'GraphVideo'].index
dfs.drop(data_sin_videos, inplace = True)

In [ ]:
#Elimimo post de tipo Sidecar.
data_sin_sidecar = dfs[dfs['tipo']== 'GraphSidecar'].index
dfs.drop(data_sin_sidecar, inplace = True)


In [ ]:
dataset = dfs[['post','likes','sponsored', 'nPost', 'followers', 'followees', 'nIgtv','business','verified', 'day', 'month', 'hour_of_day', 'category' ]]

In [ ]:
dataset.to_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/dataset_train.csv')

In [ ]:
data_faces = pd.read_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/num_faces.csv', on_bad_lines='skip', sep=',')
data_space = pd.read_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/in_out.csv', on_bad_lines='skip', sep=',')

img_c = pd.merge(left=data_faces,right=data_space, left_on='post', right_on='post')
img_c.to_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/dataset_img.csv')

In [ ]:
sep = img_c["post"].str.split('.', expand=True)
sep.columns = ['id', 'ext']
data_img = pd.concat([img_c, sep], axis=1)
data_img

In [ ]:
dataset_c = pd.merge(left=dataset,right=data_img, left_on='post', right_on='id')
dataset_c

In [ ]:
dataset_train = dataset_c[['post_x','likes','sponsored', 'nPost', 'followers', 'followees', 'nIgtv','business','verified', 'day', 'month', 'hour_of_day', 'category', 'faces', 'is_people', 'label' ]]
dataset_train.to_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/dataset_train_c.csv')

In [ ]:
dataset_train = pd.read_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/dataset_train_c.csv', on_bad_lines='skip', sep=',')
dataset_train['followers'].max()

In [ ]:
from sklearn.preprocessing import LabelEncoder
#one hot encoding
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
# definimos datos
data_day = dataset_train[['day']].values
data_month = dataset_train[['month']].values
data_hour_of_day = dataset_train[['hour_of_day']].values
data_category = dataset_train[['category']].values
data_is_people = dataset_train[['is_people']].values
data_label = dataset_train[['label']].values
# definimos one hot encoding
encoder = OneHotEncoder(sparse=False)
# transformamos datos
onehot_day = encoder.fit_transform(data_day)
onehot_month = encoder.fit_transform(data_month)
onehot_hour_of_day = encoder.fit_transform(data_hour_of_day)
onehot_category = encoder.fit_transform(data_category)
onehot_is_people = encoder.fit_transform(data_is_people)
onehot_label = encoder.fit_transform(data_label)



In [ ]:
dataset_train['day'] = onehot_day
dataset_train['month'] = onehot_month
dataset_train['hour_of_day'] = onehot_hour_of_day
dataset_train['category'] = onehot_category
dataset_train['is_people'] = onehot_is_people
dataset_train['label'] = onehot_label

In [ ]:
dataset_train.to_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/dataset_codif.csv')

In [ ]:
dataset_train.dtypes

In [ ]:
dataset_train_sin_img = dataset_c

In [ ]:
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
correlation = dataset_train_sin_img.corr(method='pearson')
columns = correlation.nlargest(10, 'likes').index
columns
correlation_map = np.corrcoef(dataset_train_sin_img[columns].values.T)
sns.set(font_scale=0.75)
heatmap = sns.heatmap(correlation_map, cbar=True, annot=True, square=True, fmt='.2f', yticklabels=columns.values, xticklabels=columns.values)

plt.show()

In [ ]:
img = '/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/num_faces.csv'
with open( '/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/num_faces.csv', 'a', encoding='UTF-8') as archivo:
  print('codificando')
  archivo.write('\n')



In [ ]:
dataFace = pd.read_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/num_faces1.csv',on_bad_lines='skip', sep=';' )
dataFace
df = dataFace["post"].str.split('.', expand=True)
df.columns = ['post_id', 'ext']
dface = pd.concat([dataFace, df], axis=1)
dface